In [1]:
import requests
import json
import re
import pandas as pd

In [2]:
sex = "Men "
course = "Yard "
diving_event1 = "F005"
diving_event2 = "F020"
relay_ev1 = "F001"
relay_ev2 = "F006"
relay_ev3 = "F007"
relay_ev4 = "F014"
relay_ev5 = "F021"

list_of_events = [
    "160225F001", "160225F002", "160225F003", "160225F004", "160225F005", "160225F006",
    "160225F007", "160225F008", "160225F009", "160225F010", "160225F011", "160225F012", "160225F013", "160225F014",
    "160225F015", "160225F016", "160225F017", "160225F018", "160225F019", "160225F020", "160225F021"
]
swimming_events = []
diving_events = []
relay_events = []
for event in list_of_events:
    if diving_event1 in event:
        diving_events.append(event)
    elif diving_event2 in event:
        diving_events.append(event)
    elif relay_ev1 in event:
        relay_events.append(event)
    elif relay_ev2 in event:
        relay_events.append(event)
    elif relay_ev3 in event:
        relay_events.append(event)
    elif relay_ev4 in event:
        relay_events.append(event)
    elif relay_ev5 in event:
        relay_events.append(event)
    else:
        swimming_events.append(event)
    
swimming_events

['160225F002',
 '160225F003',
 '160225F004',
 '160225F008',
 '160225F009',
 '160225F010',
 '160225F011',
 '160225F012',
 '160225F013',
 '160225F015',
 '160225F016',
 '160225F017',
 '160225F018',
 '160225F019']

In [3]:
event = swimming_events[1]
def get_event(event):
    event = requests.get("http://www.neswim.com/2016mensivy/"+ event + ".htm").text
    event = event.split("Event ")
    event = event[1].replace("<span></span>", "")
    event2 = event.split("\n")
    return event2

event_results = get_event(event)
event_results


['3  Men 200 Yard IM</b>',
 '=========================================================================',
 '  Meet Record: M 1:43.94  3/7/2013  Nejc Zupan (Dartmouth)',
 '  Pool Record: P 1:43.94  2013      Nejc Zupan (Dartmouth)',
 '                 1:42.51  A NCAA  A',
 '                 1:49.09  B NCAA B',
 '    Name           Year School            Prelims     Finals       Points ',
 '=========================================================================',
 '<b>                            === A - Final ===                            </b>',
 ' ',
 "  1 D'Alessandro, T  SR Princeton         1:45.27    1:44.33 B       32  ",
 '                 22.80        49.13 (26.33)',
 '        1:19.03 (29.90)     1:44.33 (25.30)',
 '  2 Bove, Marco      SR Princeton         1:45.76    1:45.66 B       28  ',
 '                 23.11        50.01 (26.90)',
 '        1:19.90 (29.89)     1:45.66 (25.76)',
 '  3 Andrew, Mark W   FR Penn              1:45.15    1:45.73 B       27  ',
 '              

In [4]:
def get_event_name_num(list_of_strings):
    string = list_of_strings[0]
    string = string.replace("</b>", "")
    print("Getting Ev#, Distance, and Stroke for " + string)
    event_num = int(string[:3])
    event_dist = string.split(sex)[1]
    event_dist = event_dist.split(" ")[0]
    event_dist = int(event_dist)
    event_stroke = string.split(course)[1]
    return event_num, event_dist, event_stroke
    
    
event_name_num = get_event_name_num(event_results)
event_name_num

Getting Ev#, Distance, and Stroke for 3  Men 200 Yard IM


(3, 200, 'IM')

In [5]:
def cut_splits(list_of_strings):
    new_list = []
    for line in list_of_strings[1:]:
        try:
            s = int(line[1:3])
            new_list.append(line)
        except ValueError:
            pass
    return new_list
        
event_results = cut_splits(event_results)
event_results

["  1 D'Alessandro, T  SR Princeton         1:45.27    1:44.33 B       32  ",
 '  2 Bove, Marco      SR Princeton         1:45.76    1:45.66 B       28  ',
 '  3 Andrew, Mark W   FR Penn              1:45.15    1:45.73 B       27  ',
 '  4 Sali, Dylan      JR Cornell           1:45.80    1:45.84 B       26  ',
 '  5 Proctor, Grant   JR Penn              1:46.60    1:46.70 B       25  ',
 '  6 Yeager, Christi  SR Harvard           1:46.42    1:47.24 B       24  ',
 '  7 Schaffer, Cj J   SO Penn              1:46.80    1:47.63 B       23  ',
 '  8 Tran, Daniel T   FR Harvard           1:47.34    1:48.06 B       22  ',
 '  9 Dillinger, Thom  FR Penn              1:47.41    1:46.18 B       20  ',
 ' 10 Sanborn, Byron   SR Princeton         1:47.61    1:46.92 B       17  ',
 ' 11 Buerger, Zach T  SO Princeton         1:47.53    1:46.95 B       16  ',
 ' 12 Park, Jae Y      SO Columbia          1:48.37    1:47.37 B       15  ',
 ' 13 Osada, Koya      SO Harvard           1:47.79    1:47.68 B

In [6]:
def extract_data_from_line_to_list(string):
    rank = int(line[:4])
    name = line[4:20]
    year = line[21:23]
    school = line[24:40].replace(" ", "")
    prelim = line[42:50].replace(" ", "")
    final = line[53:60].replace(" ", "")
    try:
        points = float(line[69:])
    except ValueError:
        points = 0
    return [rank, name, year, school, prelim, final, points]

nlist = []
for line in event_results:
    d = extract_data_from_line_to_list(line)
    nlist.append(d)
nlist

[[1, "D'Alessandro, T ", 'SR', 'Princeton', '1:45.27', '1:44.33', 32.0],
 [2, 'Bove, Marco     ', 'SR', 'Princeton', '1:45.76', '1:45.66', 28.0],
 [3, 'Andrew, Mark W  ', 'FR', 'Penn', '1:45.15', '1:45.73', 27.0],
 [4, 'Sali, Dylan     ', 'JR', 'Cornell', '1:45.80', '1:45.84', 26.0],
 [5, 'Proctor, Grant  ', 'JR', 'Penn', '1:46.60', '1:46.70', 25.0],
 [6, 'Yeager, Christi ', 'SR', 'Harvard', '1:46.42', '1:47.24', 24.0],
 [7, 'Schaffer, Cj J  ', 'SO', 'Penn', '1:46.80', '1:47.63', 23.0],
 [8, 'Tran, Daniel T  ', 'FR', 'Harvard', '1:47.34', '1:48.06', 22.0],
 [9, 'Dillinger, Thom ', 'FR', 'Penn', '1:47.41', '1:46.18', 20.0],
 [10, 'Sanborn, Byron  ', 'SR', 'Princeton', '1:47.61', '1:46.92', 17.0],
 [11, 'Buerger, Zach T ', 'SO', 'Princeton', '1:47.53', '1:46.95', 16.0],
 [12, 'Park, Jae Y     ', 'SO', 'Columbia', '1:48.37', '1:47.37', 15.0],
 [13, 'Osada, Koya     ', 'SO', 'Harvard', '1:47.79', '1:47.68', 14.0],
 [14, 'Springman, Rile ', 'SO', 'Brown', '1:48.68', '1:48.20', 13.0],
 [15, 

In [7]:
meet_results = pd.DataFrame(columns = ["Ev#", "Distance", "Stroke", "Rank", "Name", "Yr", "School", "Prelim", "Final", "Points"])

for event in swimming_events:
    event_results = get_event(event)
    event_name_num = get_event_name_num(event_results)
    event_results = cut_splits(event_results)
    nlist = []
    for line in event_results:
        d = extract_data_from_line_to_list(line)
        nlist.append(d)
    pandized_event = pd.DataFrame(nlist)
    pandized_event.columns = ["Rank", "Name", "Yr", "School", "Prelim", "Final", "Points"]
    pandized_event["Ev#"] = event_name_num[0]
    pandized_event["Distance"] = event_name_num[1]
    pandized_event["Stroke"] = event_name_num[2]
    meet_results = meet_results.append(pandized_event)

meet_results = meet_results[["Ev#", "Distance", "Stroke", "Rank", "Name", "Yr", "School", "Prelim", "Final", "Points"]]
meet_results

Getting Ev#, Distance, and Stroke for 2  Men 500 Yard Freestyle
Getting Ev#, Distance, and Stroke for 3  Men 200 Yard IM
Getting Ev#, Distance, and Stroke for 4  Men 50 Yard Freestyle
Getting Ev#, Distance, and Stroke for 8  Men 1000 Yard Freestyle
Getting Ev#, Distance, and Stroke for 9  Men 400 Yard IM
Getting Ev#, Distance, and Stroke for 10  Men 100 Yard Butterfly
Getting Ev#, Distance, and Stroke for 11  Men 200 Yard Freestyle
Getting Ev#, Distance, and Stroke for 12  Men 100 Yard Breaststroke
Getting Ev#, Distance, and Stroke for 13  Men 100 Yard Backstroke
Getting Ev#, Distance, and Stroke for 15  Men 1650 Yard Freestyle
Getting Ev#, Distance, and Stroke for 16  Men 200 Yard Backstroke
Getting Ev#, Distance, and Stroke for 17  Men 100 Yard Freestyle
Getting Ev#, Distance, and Stroke for 18  Men 200 Yard Breaststroke
Getting Ev#, Distance, and Stroke for 19  Men 200 Yard Butterfly


Ev#  Distance        Stroke  Rank              Name  Yr     School  \
0     2       500     Freestyle     1  Swanson, Chris    SR       Penn   
1     2       500     Freestyle     2  Hyogo, Kei        SO       Yale   
2     2       500     Freestyle     3  Hogan, Brian R    SR       Yale   
3     2       500     Freestyle     4  Smiddy, Samuel    JR  Princeton   
4     2       500     Freestyle     5  Abdel Khalik, A   JR    Harvard   
5     2       500     Freestyle     6  Houck, Logan T    FR    Harvard   
6     2       500     Freestyle     7  Peterson, Alex    SO       Penn   
7     2       500     Freestyle     8  Novak, Brennan    FR    Harvard   
8     2       500     Freestyle     9  Boyd, Jack P      SO    Harvard   
9     2       500     Freestyle    10  Lin, Adrian Y     FR       Yale   
10    2       500     Freestyle    11  Bondarenko, Nik   JR   Columbia   
11    2       500     Freestyle    12  Uselis, Taylor    SO       Penn   
12    2       500     Freestyle    13  McQuet, Murphy    FR  Princeton   
13    2       500     Freestyle    14  Bole, Scott M     FR       Yale   
14    2       500     Freestyle    15  Ginsborg, Josh    SR       Yale   
15    2       500     Freestyle    16  Zyla, Michal A    SO   Columbia   
16    2       500     Freestyle    17  Haeffner, Kent    SO    Harvard   
17    2       500     Freestyle    18  Sweezer, Brando   SO    Cornell   
18    2       500     Freestyle    19  Ridout, Zachary   SR  Princeton   
19    2       500     Freestyle    20  Sharkey, Ryan P   SO    Cornell   
20    2       500     Freestyle    20  Jacobs, Talbot    SO      Brown   
21    2       500     Freestyle    22  Lerude, Ben E     JR       Yale   
22    2       500     Freestyle    23  Mayfield, Cory    SR      Brown   
23    2       500     Freestyle    24  Kolyvek, Koppi    FR    Cornell   
24    2       500     Freestyle    25  Casey, Grant C    SO      Brown   
25    2       500     Freestyle    26  Treil, Alexande   SR      Brown   
26    2       500     Freestyle    27  Jacobsen, Carte   FR  Dartmouth   
27    2       500     Freestyle    28  Schnaars, Georg   SR    Cornell   
28    2       500     Freestyle    29  Ng, Eric D        FR   Columbia   
29    2       500     Freestyle    30  Bell, Dirk D      SO       Yale   
..  ...       ...           ...   ...               ...  ..        ...   
21   18       200  Breaststroke    22  Hall, Delaney W   FR  Dartmouth   
22   18       200  Breaststroke    23  Crystal, Brenda   SR       Penn   
23   18       200  Breaststroke    24  Limaldi, Jacob    FR       Yale   
24   18       200  Breaststroke    25  Vaimann, Timo     JR  Dartmouth   
25   18       200  Breaststroke    26  Edmundson, Iain   FR  Dartmouth   
0    19       200     Butterfly     1  Luna, Jacob K     SR    Harvard   
1    19       200     Butterfly     2  Okubo, Corey H    SO  Princeton   
2    19       200     Butterfly     3  Bove, Marco       SR  Princeton   
3    19       200     Butterfly     4  Springman, Rile   SO      Brown   
4    19       200     Butterfly     5  Buese, Cole C     FR  Princeton   
5    19       200     Butterfly     6  Carbone, Joey J   FR       Yale   
6    19       200     Butterfly     7  Yakubovich, Max   JR    Harvard   
7    19       200     Butterfly     8  Buerger, Zach T   SO  Princeton   
8    19       200     Butterfly     9  Treil, Alexande   SR      Brown   
9    19       200     Butterfly    10  St John, Carl A   SR    Cornell   
10   19       200     Butterfly    11  Brakovec, Hunte   SO       Penn   
11   19       200     Butterfly    12  Declercq, Vince   SO    Cornell   
12   19       200     Butterfly    13  Mango, Alex V     SR   Columbia   
13   19       200     Butterfly    14  Russiello, Ian    FR    Cornell   
14   19       200     Butterfly    15  Brett, Shane      FR   Columbia   
15   19       200     Butterfly    16  Foster, Jack A    JR   Columbia   
16   19       200     Butterfly    17  Harmon, David B   JR  Dartmouth   
17   19 

In [8]:
meet_results.to_csv("ind_swim_event_results.csv")

In [24]:
event = relay_events[3]
print(event)

def get_relay_event(event):
    event = requests.get("http://www.neswim.com/2016mensivy/"+ event + ".htm").text
    event = event.split("Event ")
    event = event[1].replace("<span></span>", "")
    event2 = event.split("\n")
    return event2

relay_event = get_relay_event(event)
relay_event

160225F014


['14  Men 800 Yard Freestyle Relay</b>',
 '==================================================================================',
 '  Meet Record: M 6:24.75  2009      Princeton',
 '                          D.Lennox, C.Cordes, T.McNamara, J.Hartmann',
 '  Pool Record: P 6:26.87  2013      Princeton',
 "                          D'Alessandro, Maher, Wong, Bove,",
 '                 6:21.54  A NCAA  A',
 '                 6:25.06  B NCAA B',
 '    School                           Seed     Finals       Points ',
 '==================================================================================',
 "  1 Princeton  'A'                6:27.23    6:25.73P        64  ",
 '     1) Schafer, Ben C SO             2) Mackrel, Julian A JR         ',
 "     3) D'Alessandro, Teo A SR        4) Bole, Sandy A SR             ",
 '                 21.86        45.99 (45.99)',
 '      1:10.81 (1:10.81)   1:37.22 (1:37.22)',
 '        1:58.65 (21.43)     2:23.32 (46.10)',
 '      2:48.92 (1:11.70)   3:13.90

In [25]:
def get_relay_name_num(list_of_strings):
    string = list_of_strings[0]
    string = string.replace("</b>", "")
    print("Getting Ev#, Distance, and Stroke for " + string)
    event_num = int(string[:3])
    event_dist = string.split(sex)[1]
    event_dist = event_dist.split(" ")[0]
    event_dist = int(event_dist)
    event_stroke = string.split(course)[1]
    return event_num, event_dist, event_stroke

relay_name_num = get_relay_name_num(relay_event)
relay_name_num

Getting Ev#, Distance, and Stroke for 14  Men 800 Yard Freestyle Relay


(14, 800, 'Freestyle Relay')

In [26]:
def cut_relay_splits(list_of_strings):
    new_list = []
    for line in list_of_strings:
        try:
            s = int(line[1:3])
            new_list.append(line)
        except ValueError:
            pass
    return new_list
        
relay_results = cut_relay_splits(relay_event)
relay_results

['14  Men 800 Yard Freestyle Relay</b>',
 "  1 Princeton  'A'                6:27.23    6:25.73P        64  ",
 "  2 Penn  'A'                     6:27.73    6:25.79P        56  ",
 "  3 Yale  'A'                     6:34.13    6:26.01P        54  ",
 "  4 Harvard  'A'                  6:32.81    6:31.17         52  ",
 "  5 Brown  'A'                    6:46.03    6:40.41         50  ",
 "  6 Dartmouth  'A'                6:42.02    6:41.64         48  ",
 "  7 Cornell  'A'                  6:42.03    6:42.45         46  "]

In [55]:
def extract_RELAY_data_from_line_to_list(line):
    try:
        rank = int(line[:2])
    except ValueError:
        rank = "NaN"
        print("NaN yall" + line)
    name = "NaN"
    year = "NaN"
    school = line[2:].split("'A'")[0].replace(" ", "")
    prelim = line.split("'A'")[1].replace(" ", "")[:7]
    final = line.split("'A'")[1].replace(" ", "")[7:14]
    try:
        points = float(line[58:])
    except ValueError:
        points = 0
    return [rank, name, year, school, prelim, final, points]

nlist = []
for line in relay_results[1:]:
    d = extract_RELAY_data_from_line_to_list(line[2:])
    nlist.append(d)
nlist

[[1, 'NaN', 'NaN', 'Princeton', '6:27.23', '6:25.73', 64.0],
 [2, 'NaN', 'NaN', 'Penn', '6:27.73', '6:25.79', 56.0],
 [3, 'NaN', 'NaN', 'Yale', '6:34.13', '6:26.01', 54.0],
 [4, 'NaN', 'NaN', 'Harvard', '6:32.81', '6:31.17', 52.0],
 [5, 'NaN', 'NaN', 'Brown', '6:46.03', '6:40.41', 50.0],
 [6, 'NaN', 'NaN', 'Dartmouth', '6:42.02', '6:41.64', 48.0],
 [7, 'NaN', 'NaN', 'Cornell', '6:42.03', '6:42.45', 46.0]]

In [50]:
for event in relay_events:
    event_results = get_relay_event(event)
    event_name_num = get_relay_name_num(event_results)
    event_results = cut_relay_splits(event_results)
    nlist = []
    for line in event_results:
        print(d)
        d = extract_RELAY_data_from_line_to_list(line[2:])
        nlist.append(d)
    pandized_event = pd.DataFrame(nlist)
    pandized_event.columns = ["Rank", "Name", "Yr", "School", "Prelim", "Final", "Points"]
    pandized_event["Ev#"] = event_name_num[0]
    pandized_event["Distance"] = event_name_num[1]
    pandized_event["Stroke"] = event_name_num[2]
    meet_results = meet_results.append(pandized_event)



Getting Ev#, Distance, and Stroke for 1  Men 200 Yard Freestyle Relay
[7, 'NaN', 'NaN', 'Cornell', ':42.036', ':42.454', 46.0]
[1, 'NaN', 'NaN', 'Harvard', ':21.121', ':17.55P', 64.0]
[2, 'NaN', 'NaN', 'Princeton', ':18.831', ':17.66P', 56.0]
[3, 'NaN', 'NaN', 'Penn', ':19.771', ':19.495', 54.0]
[4, 'NaN', 'NaN', 'Cornell', ':21.281', ':19.915', 52.0]
[5, 'NaN', 'NaN', 'Columbia', ':23.531', ':19.935', 50.0]
[6, 'NaN', 'NaN', 'Dartmouth', ':22.691', ':20.564', 48.0]
[7, 'NaN', 'NaN', 'Brown', ':22.211', ':21.114', 46.0]
Getting Ev#, Distance, and Stroke for 6  Men 400 Yard Medley Relay
[8, 'NaN', 'NaN', 'Yale', ':23.161', ':23.054', 44.0]
[1, 'NaN', 'NaN', 'Princeton', ':11.243', ':09.20M', 64.0]
[2, 'NaN', 'NaN', 'Harvard', ':15.183', ':10.56P', 56.0]
[3, 'NaN', 'NaN', 'Cornell', ':18.713', ':12.225', 54.0]
[4, 'NaN', 'NaN', 'Penn', ':12.573', ':12.265', 52.0]
[5, 'NaN', 'NaN', 'Columbia', ':20.933', ':14.485', 50.0]
[6, 'NaN', 'NaN', 'Dartmouth', ':17.273', ':14.704', 48.0]
[7, 'NaN'

IndexError: list index out of range

In [ ]:
meet_results